### Replies scraper

In [4]:
!pip install webdriver_manager

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.firefox import GeckoDriverManager

from time import sleep

In [6]:
target_tweet='https://twitter.com/ShowmaxNG/status/1735057012252442931'

In [7]:
# Twitter Login 
twitter_usr="@iam_UmarMB"
twitter_pass='0903048656

def twitter_login(driver, twitter_usr=str, twitter_pass=str):
    driver.get('https://twitter.com/i/flow/login')
    sleep(6)
    user = driver.find_element(by=By.XPATH, value='//*[@autocomplete="username"]')
    sleep(1)
    user.send_keys(twitter_usr)
    sleep(1)
    next_btn = driver.find_element(by=By.XPATH, value='/html/body/div/div/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div/div[6]/div')
    next_btn.click()
    sleep(4)
    psswd_in = driver.find_element(by=By.XPATH, value='//*[@autocomplete="current-password"]')
    psswd_in.send_keys(twitter_pass)
    sleep(2)
    login_btn = driver.find_element(by=By.XPATH, value='//html/body/div/div/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div')
    login_btn.click()
    sleep(3)
    print('Login Successful')
    return driver


In [9]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep

# Twitter Login
twitter_usr = "@iam_UmarMB"
twitter_pass = 'Ba19ba940326'

def twitter_login(driver, twitter_usr=str, twitter_pass=str):
    driver.get('https://twitter.com/i/flow/login')

    # Wait for the username input field to be present
    user = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@autocomplete="username"]'))
    )
    user.send_keys(twitter_usr)
    sleep(1)

    # Click on the next button
    next_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div/div[6]/div'))
    )
    next_btn.click()
    sleep(4)

    # Wait for the password input field to be present
    psswd_in = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@autocomplete="current-password"]'))
    )
    psswd_in.send_keys(twitter_pass)
    sleep(2)

    # Click on the login button
    login_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//html/body/div/div/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div'))
    )
    login_btn.click()
    sleep(3)

    print('Login Successful')
    return driver

# Set up the driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Call the login function
twitter_login(driver, twitter_usr=twitter_usr, twitter_pass=twitter_pass)


Login Successful


<selenium.webdriver.chrome.webdriver.WebDriver (session="5c4711986d140d1c1566e546aa01289e")>

In [8]:
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()))

twitter_login(driver, twitter_usr=twitter_usr, twitter_pass=twitter_pass)

NoSuchElementException: Message: Unable to locate element: //*[@autocomplete="current-password"]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16


In [12]:
import time

tweets = []

driver.get(target_tweet)

time.sleep(6)

MAX_SCROLLS=5
for _ in range(MAX_SCROLLS):
    last = driver.find_elements(By.XPATH, '//div[@data-testid="cellInnerDiv"]')[-1]
    driver.execute_script("arguments[0].scrollIntoView(true)", last)
    time.sleep(.2)
    all_tweets = driver.find_elements(By.XPATH, '//div[@data-testid]//article[@data-testid="tweet"]')
    for item in all_tweets[1:]: # skip first tweet because it is BBC tweet
 
            try:
                date = item.find_element(By.XPATH, './/time').text
            except:
                date = '[empty]'

            try:
                text = item.find_element(By.XPATH, './/div[@data-testid="tweetText"]').text
            except:
                text = '[empty]'


            try:
                replying_to = item.find_element(By.XPATH, './/div[contains(text(), "Replying to")]//a').text
            except:
                replying_to = '[empty]'
            
            tweets.append([date, replying_to, text])
            time.sleep(.2)
            
print(f'Found {len(tweets)} replies.')

Found 75 replies.


In [13]:
# Scraping the replies loaded by the 'load more replies' button, if there are such. 
#The "load more replies" XPATH changes dinamically and i cannot figure out the mechanics, so for now i'm brute-forcing it.
# Filling the missing values with None 

for i in range(20):
    try:
        show_more=driver.find_element(By.XPATH, value=f'/html/body/div[1]/div/div/div[2]/main/div/div/div/div[1]/div/section/div/div/div[{i}]/div/div/div/div/div/span')
        show_more.click()
        print('Found more replies!')
        for _ in range(MAX_SCROLLS):
            last = driver.find_elements(By.XPATH, '//div[@data-testid="cellInnerDiv"]')[-1]
            driver.execute_script("arguments[0].scrollIntoView(true)", last)
            time.sleep(.2)
            all_tweets = driver.find_elements(By.XPATH, '//div[@data-testid]//article[@data-testid="tweet"]')
            for item in all_tweets: # skip first tweet because it is BBC tweet

                    try:
                        date = item.find_element(By.XPATH, './/time').text
                    except:
                        date = None

                    try:
                        text = item.find_element(By.XPATH, './/div[@data-testid="tweetText"]').text
                    except:
                        text = None

                    try:
                        replying_to = item.find_element(By.XPATH, './/div[contains(text(), "Replying to")]//a').text
                    except:
                        replying_to = None

                    tweets.append([date, replying_to, text])
                    time.sleep(.2)

        print(f'Found {len(tweets)} replies totally.')
        driver.quit()
    except:
        continue

In [14]:
# Example of dataframe construction
import pandas as pd

df = pd.DataFrame(tweets, columns=['Date of Tweet', 'Replying to', 'Tweet'])


In [15]:
df

,Date of Tweet,Replying to,Tweet
0,Dec 14,[empty],Is it on YouTube??
1,Dec 14,[empty],"No it is not, it is streaming on Showmax ~AA"
2,Dec 14,[empty],I love showmax
3,Dec 14,[empty],We love you too #DezemberWithShowmax ~FO
4,[empty],[empty],"Strategy, RPG, MMO and endless action — see wh..."
...,...,...,...
70,Dec 14,[empty],Wura season has kicked off and I’m enjoying it
71,Dec 19,[empty],"I’m in love with this movie already , locked i..."
72,Dec 14,[empty],This one where kanayo day hmmmm
73,Dec 14,[empty],@kudahelp_ng\n \n@joinkuda\n should reverse my...


## TO DO